# Руководство по сериализации и валидации данных в FastAPI (Pydantic)

Ниже представлена обновлённая версия материала, в которой оставлены только главы 1 и 6 из предыдущего руководства, а также добавлены подробные примеры кода, демонстрирующие механизмы сериализации и десериализации в Pydantic. Теперь каждый пример содержит более детальные комментарии.

---

## 1. Сериализация данных в FastAPI

### 1.1. Что такое сериализация?

**Сериализация** — это процесс превращения структур данных или объектов (например, объектов Python) в формат, который может передаваться по сети (чаще всего JSON) и обратно. Например, когда клиент отправляет JSON на сервер, сервер должен превратить этот JSON во внутреннюю модель Python, чтобы с ней работать. А при ответе — сервер сериализует модель обратно в JSON.

**Зачем нужна сериализация?**

1. **Удобный обмен данными** между разными сервисами или клиентами, написанными на разных языках.
2. **Стандартизированный формат** (JSON) облегчает интеграцию.
3. **Валидация и контроль**: перед сохранением в базу данных мы можем проверить корректность входных данных.

### 1.2. Зачем нам Pydantic?

В FastAPI за сериализацию и валидацию данных отвечает **Pydantic**. Он позволяет:

- Описывать структуры данных с помощью классов, наследованных от `BaseModel`.
- Автоматически валидировать входящие данные согласно типам (например, `str`, `int`, `EmailStr`, `conint` и т. д.).
- Удобно работать с JSON: конвертировать модель в словарь, а потом в JSON, и обратно.
- Перехватывать и обрабатывать ошибки (например, если поле `email` должно быть корректным адресом).

### 1.3. Дополнительные возможности Pydantic

Pydantic не ограничивается базовой проверкой типов. Есть и более продвинутые инструменты:

1. **Валидация значений**. Например, поле может иметь ограничения:
   ```python
   from pydantic import BaseModel, conint

   class Product(BaseModel):
       name: str
       price: conint(gt=0)  # цена должна быть > 0
   ```
   Если передать `price=0`, будет ошибка.

2. **Значения по умолчанию**:
   ```python
   from typing import Optional

   class Article(BaseModel):
       title: str
       content: str
       published: bool = False  # если не указать, то будет False
   ```

3. **Опциональные поля**:
   ```python
   class Comment(BaseModel):
       user_id: int
       text: str
       parent_comment_id: Optional[int] = None
   ```
   Здесь `parent_comment_id` может быть `None`, если это корневой комментарий.

4. **Вложенные модели**:
   ```python
   from pydantic import BaseModel, EmailStr

   class Author(BaseModel):
       name: str
       email: EmailStr

   class Book(BaseModel):
       title: str
       author: Author
   ```
   Pydantic позволит создавать более сложные структуры, проверяя каждую вложенную модель.

5. **Кастомные валидаторы**:
   Вы можете определять методы (`@validator` или `@field_validator` в Pydantic v2) внутри класса для проверки полей. Например, проверить сложность пароля, валидность даты и т. д.

---

## 2. Примеры сериализации в Pydantic

Ниже — несколько примеров кода, в которых демонстрируются различные приёмы и фишки работы с Pydantic при **сериализации** данных. Каждый пример можно скопировать в отдельный файл и запустить.

### 01_dict_to_json.py

In [7]:
from pydantic import BaseModel


class User(BaseModel):
    id: int
    name: str
    email: str | None = None


# TODO-1: поменяйте значение name:  "Alice" -> 100. Запустите код.
# TODO-2: поменяйте значение id:  123 -> "123". Запустите код.
user_data = {"id": 123, "name": "Alice", "email": "alice@example.com"}
# Создаём модель, используя распаковку словаря (**user_data)
user = User(**user_data)

# Сериализуем модель в JSON-строку с отступами
user_json = user.model_dump_json(indent=4)
# Выводим результат на экран
print(user_json)

{
    "id": 123,
    "name": "Alice",
    "email": "alice@example.com"
}


**Комментарии:**
1. `class User(BaseModel): ...` — создаём класс-модель, описывающий поля.
2. В `user_data` задаём поле `id`, поле `name`, а `email` необязательное.
3. Если нарушить типизацию, например, передать `name=100`, Pydantic выбросит ошибку.
4. `model_dump_json(indent=4)` делает красивый форматированный JSON.

---

### 02_object_to_json.py

In [47]:
from pydantic import BaseModel, ConfigDict


class Point:
    def __init__(self, x, y):
        self.x = x
        self.y = y


class Point3D:
    def __init__(self, x, y, z):
        self.x = x
        self.y = y
        self.z = z

class Point1D:
    def __init__(self, x):
        self.x = x


class PointSchema(BaseModel):
    # ConfigDict(from_attributes=True) позволяет брать атрибуты из обычных Python-объектов
    model_config = ConfigDict(from_attributes=True, extra='allow')
    x: int
    y: int



# TODO-1: поменяйте значение координат: int -> float
# TODO-2: поменяйте значение координат: int -> str
print(PointSchema.model_config)
point = Point(x=10, y=25)
# Создаём из объекта point модель PointSchema
schema = PointSchema.model_validate(point)
print(f'{schema = }')
# Сериализуем результат в JSON
point_json = schema.model_dump_json(indent=4)

print(point_json)


# TODO-3: создаём экземпляр класса Point3D и сериализуем
point3d = Point3D(x=1, y=2, z=3)
try:
    schema_3d = PointSchema.model_validate(point3d)
    point3d_json = schema_3d.model_dump_json(indent=4)
    print("Point3D:\n", point3d_json)
except Exception as e:
    print("Validation error for Point3D:", e)


# TODO-4: создаём экземпляр класса Point1D и сериализуем
# point1d = Point1D(x=100)
# try:
#     schema_1d = PointSchema.model_validate(point1d)
#     point1d_json = schema_1d.model_dump_json(indent=4)
#     print("Point1D:\n", point1d_json)
# except Exception as e:
#     print("Validation error for Point1D:", e)

{'from_attributes': True, 'extra': 'allow'}
schema = PointSchema(x=10, y=25)
{
    "x": 10,
    "y": 25
}
Point3D:
 {
    "x": 1,
    "y": 2
}


**Комментарии:**
1. `PointSchema` ждёт поля `x` и `y`, оба типа `int`.
2. Если вы передадите объект `Point3D`, в котором есть ещё и `z`, это вызовет ошибку, так как в модели нет такого поля.
3. `model_config = ConfigDict(from_attributes=True)` даёт возможность передавать обычные объекты (`point`), а не словари.
4. `model_validate(...)` (в Pydantic v2) строит модель на основе объекта.

---

### 03_object_to_json_optional.py

In [19]:
from pydantic import BaseModel, ConfigDict


class Point:
    def __init__(self, x, y):
        self.x = x
        self.y = y


class Point3D:
    def __init__(self, x, y, z):
        self.x = x
        self.y = y
        self.z = z

class Point1D:
    def __init__(self, x):
        self.x = x


class PointSchema(BaseModel):
    model_config = ConfigDict(from_attributes=True)
    x: int
    y: int = None
    z: int = None


point1 = Point1D(x=10)
point2 = Point(x=10, y=25)
point3 = Point3D(x=10, y=25, z=11)


# Сериализация со значением по умолчанию
print(PointSchema.model_validate(point1).model_dump_json(indent=4))
print(PointSchema.model_validate(point2).model_dump_json(indent=4))
print(PointSchema.model_validate(point3).model_dump_json(indent=4))

{
    "x": 10,
    "y": null,
    "z": null
}
{
    "x": 10,
    "y": 25,
    "z": null
}
{
    "x": 10,
    "y": 25,
    "z": 11
}


**Комментарии:**
1. Поле `y` имеет тип `int`, но по умолчанию `None`, а `z` — тоже `int` или `None`.
2. `point1` имеет только `x`, но Pydantic не будет ругаться, потому что `y` и `z` необязательны.
3. При сериализации (метод `model_dump_json`) те поля, что не заданы, будут отсутствовать или иметь `null` в JSON.

---

### 04_nested_object_to_json.py

In [21]:
from pydantic import BaseModel


class Address(BaseModel):
    street: str
    city: str
    zip: str

class Order(BaseModel):
    order_id: int
    items: list[str]

class User(BaseModel):
    user_id: int
    username: str
    address: Address
    orders: list[Order]


user_data = {
    "user_id": 2,
    "username": "John",
    "address": {
        "street": "Main St",
        "city": "Anytown",
        "zip": "12345"
    },
    "orders": [
        {"order_id": 1, "items": ["item1", "item2"]},
        {"order_id": 2, "items": ["item3"]}
    ]
}

user = User(**user_data)
user_json = user.model_dump_json(indent=4)
print(type(user_json))
print(user_json)

<class 'str'>
{
    "user_id": 2,
    "username": "John",
    "address": {
        "street": "Main St",
        "city": "Anytown",
        "zip": "12345"
    },
    "orders": [
        {
            "order_id": 1,
            "items": [
                "item1",
                "item2"
            ]
        },
        {
            "order_id": 2,
            "items": [
                "item3"
            ]
        }
    ]
}


**Комментарии:**
1. `User` содержит поле `address` (типа `Address`) и `orders` (список `Order`).
2. `user_data` имеет вложенную структуру: объект `address` и массив `orders`.
3. Pydantic автоматически сериализует всё дерево моделей в JSON.

---

### 05_custom_validator.py

In [22]:
from pydantic import BaseModel, field_validator


class User(BaseModel):
    id: int
    name: str
    email: str

    @field_validator("email")
    def validate_email(cls, value):
        if "@" not in value:
            raise ValueError("Email должен содержать символ '@'")
        return value


user = User(id=1, name="Иван", email="ivan@example.com")
print(user.model_dump_json())

try:
    User(id=2, name="Петр", email="petrexample.com")
except ValueError as e:
    print(e)

{"id":1,"name":"Иван","email":"ivan@example.com"}
1 validation error for User
email
  Value error, Email должен содержать символ '@' [type=value_error, input_value='petrexample.com', input_type=str]
    For further information visit https://errors.pydantic.dev/2.11/v/value_error


**Комментарии:**
1. `@field_validator("email")` — специальный декоратор, который вызывается при валидации поля `email`.
2. Если значение не содержит `@`, выбрасывается ошибка `ValueError`.
3. При корректном вводе модель успешно создаётся, и мы можем вызвать `model_dump_json()`.

---

### 06_aliases.py

In [24]:
from pydantic import BaseModel, Field


class User(BaseModel):
    user_id: int = Field(alias="id")
    name: str = Field(alias="user_name")
    email: str = None

user_data = {"id": 1, "user_name": "Иван", "email": "ivan@example.com"}
user_data2 = {"id": 2, "user_name": "Петр", "email": "petr@example.com"}

user = User(**user_data)
print(f'{user = }')
print(user.model_dump_json(by_alias=True))

user_deserialized = User(**user_data2)
print(user_deserialized)

user = User(user_id=1, name='Иван', email='ivan@example.com')
{"id":1,"user_name":"Иван","email":"ivan@example.com"}
user_id=2 name='Петр' email='petr@example.com'


**Комментарии:**
1. `Field(alias="id")` означает, что Pydantic при деструктуризации словаря будет искать ключ `"id"`, а сохранять его в поле `user_id`.
2. `model_dump_json(by_alias=True)` вернёт JSON, где поле `user_id` будет записано как `"id"`.

---

## 3. Примеры десериализации

Теперь рассмотрим примеры, показывающие десериализацию (превращение JSON или словарей в Python-объекты).

### 01_dict_to_object.py

In [27]:
from pydantic import BaseModel


class User(BaseModel):
    id: int
    name: str
    age: int


user_data = {
    "id": 1,
    "name": "Иван",
    "age": 30
}

# Десериализация из словаря
user = User(**user_data)

print(user)
print(user.name)

id=1 name='Иван' age=30
Иван


**Комментарии:**
1. Мы передаём словарь `user_data` распаковкой `**user_data`.
2. Pydantic проверяет соответствие типов и создаёт объект `User`.
3. Если бы не было поля `age` или был бы неверный тип, возникла бы ошибка валидации.

---

### 02_json_to_object.py

In [30]:
from pydantic import BaseModel

class User(BaseModel):
    id: int
    name: str
    age: int

user_json = '{"id": 1, "name": "Иван", "age": 30}'
user_json_bad = '{"id": 1, "name": "Иван"}'

# Десериализация из JSON-строки
# TODO-1: попробуйте десериализовать user_json_bad
user = User.model_validate_json(user_json)

print(user)
print(user.name)

id=1 name='Иван' age=30
Иван


**Комментарии:**
1. `user_json` — валидная JSON-строка со всеми тремя полями.
2. `user_json_bad` — там нет поля `age`. Попробуйте десериализовать и увидеть ошибку.
3. `User.model_validate_json(...)` (Pydantic v2) берёт JSON-строку, парсит её и проверяет структуру.

---

### 03_nested_objects.py

In [35]:
from pydantic import BaseModel
from typing import List, Optional

class Address(BaseModel):
    street: str
    city: str
    zip_code: str

class Company(BaseModel):
    name: str
    address: Address
    employees: Optional[List[str]] = None

class Employee(BaseModel):
    id: int
    name: str
    company: Company

employee_data = {
    "id": 1,
    "name": "Алексей",
    "company": {
        "name": "ООО Рога и Копыта",
        "address": {
            "street": "Тверская",
            "city": "Москва",
            "zip_code": "123456"
        },
       "employees": ["Иван", "Петр"]
    }
}

# Десериализация из словаря с вложенными объектами
employee = Employee(**employee_data)

print(employee)
print(employee.company.address.city)

id=1 name='Алексей' company=Company(name='ООО Рога и Копыта', address=Address(street='Тверская', city='Москва', zip_code='123456'), employees=['Иван', 'Петр'])
Москва


**Комментарии:**
1. Модель `Employee` содержит вложенную модель `Company`, а `Company` — вложенную модель `Address`.
2. При создании `Employee` мы передаём вложенный словарь, и Pydantic рекурсивно проверяет/создаёт все вложенные объекты.
3. `print(employee.company.address.city)` даёт доступ к полям самой глубины.

---

### 04_list_objects.py

In [38]:
from pydantic import BaseModel
from typing import List


class User(BaseModel):
    id: int
    name: str
    age: int


users_data: list = [
    {"id": 1, "name": "Иван", "age": 30},
    {"id": 2, "name": "Петр", "age": 25}
]

# Десериализация списка словарей
users = [User(**user_data) for user_data in users_data]

for user in users:
    print(user)

id=1 name='Иван' age=30
id=2 name='Петр' age=25


**Комментарии:**
1. `users_data` — это список словарей.
2. Мы итерируемся по списку, создавая модель `User` для каждого словаря.
3. Если один из элементов будет с неправильной структурой, выбросится ошибка.

---

## 4. Заключение

Теперь вы знаете:

1. **Что такое сериализация** и почему JSON.
2. **Как Pydantic упрощает работу** с данными (включая сложные структуры, валидацию значений, вложенные модели и т.д.).

В итоге, применяя эти примеры в реальных проектах, вы сможете надёжно проверять входящие данные, управлять структурой JSON и облегчать обмен данными между сервисами.

🚀 Удачи в освоении FastAPI и Pydantic!
